# HT2 - DL

## Autores

- Angel Higueros
- Fredy Velasquez

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import time

## Task 1

In [18]:
# Task 1 - Preparación del conjunto de datos

# Cargar el dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Estandarizar y dividir en conjuntos de entrenamiento y prueba
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir a tensores de PyTorch
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

## Task 2 

In [19]:
# Task 2: Arquitectura del modelo
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Task 3, 4, 5

In [20]:
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=100):
    train_losses = []
    test_losses = []
    train_accs = []
    test_accs = []
    train_f1s = []
    test_f1s = []
    
    for epoch in range(epochs):
        # Entrenamiento
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            # Evaluación
            model.eval()
            
            # Métricas de entrenamiento
            train_loss = criterion(outputs, y_train).item()
            train_losses.append(train_loss)
            
            _, predicted = torch.max(outputs, 1)
            train_acc = accuracy_score(y_train, predicted)
            train_accs.append(train_acc)
            
            train_f1 = f1_score(y_train, predicted, average='macro')
            train_f1s.append(train_f1)
            
            # Métricas de prueba
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test).item()
            test_losses.append(test_loss)
            
            _, test_predicted = torch.max(test_outputs, 1)
            test_acc = accuracy_score(y_test, test_predicted)
            test_accs.append(test_acc)
            
            test_f1 = f1_score(y_test, test_predicted, average='macro')
            test_f1s.append(test_f1)
            
    return train_losses, test_losses, train_accs, test_accs, train_f1s, test_f1s

## Task 6

In [17]:
for loss_function in loss_functions:
    for opt in optimizers:
        model = SimpleNN(4, 10, 3)
        criterion = loss_function
        
        if opt == 'SGD':
            optimizer = optim.SGD(model.parameters(), lr=0.01)
        elif opt == 'Adam':
            optimizer = optim.Adam(model.parameters(), lr=0.01)
        elif opt == 'AdamW':
            optimizer = optim.AdamW(model.parameters(), lr=0.01)

        start_time = time.time()
        train_losses, test_losses, train_accs, test_accs, train_f1s, test_f1s = train_model(
            model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=100)
        end_time = time.time()

        elapsed_time = end_time - start_time
        
        # Visualización
        plt.figure(figsize=(15, 12))

        # Curvas de pérdida
        plt.subplot(3, 1, 1)
        plt.plot(train_losses, label='Entrenamiento', color='blue')
        plt.plot(test_losses, label='Validación', color='red')
        plt.title(f'Pérdida durante el entrenamiento - Función de pérdida: {type(loss_function).__name__}, Optimizador: {opt}')
        plt.xlabel('Época')
        plt.ylabel('Pérdida')
        plt.legend()

        # Curvas de precisión
        plt.subplot(3, 1, 2)
        plt.plot(train_accs, label='Entrenamiento', color='blue')
        plt.plot(test_accs, label='Validación', color='red')
        plt.title(f'Precisión durante el entrenamiento - Función de pérdida: {type(loss_function).__name__}, Optimizador: {opt}')
        plt.xlabel('Época')
        plt.ylabel('Precisión')
        plt.legend()

        # Curvas de F1
        plt.subplot(3, 1, 3)
        plt.plot(train_f1s, label='Entrenamiento', color='blue')
        plt.plot(test_f1s, label='Validación', color='red')
        plt.title(f'F1 durante el entrenamiento - Función de pérdida: {type(loss_function).__name__}, Optimizador: {opt}')
        plt.xlabel('Época')
        plt.ylabel('F1')
        plt.legend()

        plt.tight_layout()
        plt.show()

/Users/fredyvelasquez/anaconda3/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([120])) that is different to the input size (torch.Size([120, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (3) must match the size of tensor b (120) at non-singleton dimension 1